**Methdology** 
1. Bag of Words
2. Bag of Words with Term Frequency-Inverse Document Frequency
3. Doc2vec Log Event Analysis (Event Based)



In [1]:
# import required packages
import re
import os
import numpy as np
import pandas as pd
import hashlib
from datetime import datetime
import string
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
from textblob import Word
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from gensim.models.doc2vec import TaggedDocument
import spacy  # For preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from sklearn import utils
from sklearn.metrics import accuracy_score, f1_score
from gensim.models import Doc2Vec
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

/Users/yuxuancui/opt/miniconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/yuxuancui/opt/miniconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


**Data processing**

In [ ]:
# get the data

df = pd.read_csv('/content/drive/MyDrive/project 585/log_with_event_desc.csv')
df.head()

In [ ]:
label=pd.read_csv('/content/drive/MyDrive/project 585/anomaly_label.csv')
label['BlockId2'] = label.BlockId.str.extract('(\d+)') 
label.drop('BlockId', axis=1, inplace=True)

In [ ]:
label.BlockId2 = pd.to_numeric(label.BlockId2, errors='coerce').fillna(0).astype(np.int64)

In [ ]:
df_event = df.groupby('BlockId')['Event_Description','Content'].apply(sum)
df_event.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Event_Description,Content
BlockId,,
24807389905763,"('BLOCK', 'NameSystem', 'allocateBlock', 'user...",BLOCK* NameSystem.allocateBlock: /user/root/so...
27644297472372,"('Receiving', 'block', 'blk', 'src', 'dest')('...",Receiving block blk_27644297472372 src: /10.25...
33380551139732,"('BLOCK', 'NameSystem', 'allocateBlock', 'user...",BLOCK* NameSystem.allocateBlock: /user/root/ra...
35641716531335,"('BLOCK', 'NameSystem', 'allocateBlock', 'user...",BLOCK* NameSystem.allocateBlock: /user/root/ra...
40083153975957,"('Receiving', 'block', 'blk', 'src', 'dest')('...",Receiving block blk_40083153975957 src: /10.25...


In [ ]:
log = pd.merge(df_event, label, left_on='BlockId', right_on='BlockId2', how='left').drop('BlockId2', axis=1)
log.head()


,Event_Description,Content,Label
0,"('BLOCK', 'NameSystem', 'allocateBlock', 'user...",BLOCK* NameSystem.allocateBlock: /user/root/so...,Normal
1,"('Receiving', 'block', 'blk', 'src', 'dest')('...",Receiving block blk_27644297472372 src: /10.25...,Normal
2,"('BLOCK', 'NameSystem', 'allocateBlock', 'user...",BLOCK* NameSystem.allocateBlock: /user/root/ra...,Normal
3,"('BLOCK', 'NameSystem', 'allocateBlock', 'user...",BLOCK* NameSystem.allocateBlock: /user/root/ra...,Normal
4,"('Receiving', 'block', 'blk', 'src', 'dest')('...",Receiving block blk_40083153975957 src: /10.25...,Normal


In [ ]:
len(log)
log.to_csv('/content/drive/MyDrive/586_project_NPL/log_label.csv')

In [ ]:
Content_npl=[]
for line in log.Content:
  # extract only character add description of the event 
  match=re.findall('[a-zA-Z]+',str(line))
  content=re.sub("[^a-zA-Z]+",' ',str(line))
  Content_npl.append(content)

In [ ]:
log['Content_npl']=Content_npl

In [ ]:
log_rnn=log[["Label","Content_npl"]]

In [ ]:
log_rnn.to_csv('/content/drive/MyDrive/586_project_NPL/log_rnn.csv')

In [ ]:
log_rnn.groupby('Label').count()

,Content_npl
Label,
Anomaly,16838
Normal,558223


***1. Bag of Words***. events

In [6]:
#data=pd.read_csv('/content/drive/MyDrive/586_project_NPL/log_rnn.csv')
data=pd.read_csv('/Users/yuxuancui/Desktop/MDS/data586/project/rnn/unique_id.csv')

In [7]:
data.head()

,Unnamed: 0,BlockId,Label,Event,Event_id
0,0,24807389905763,Normal,"['E10', 'E31', 'E31', 'E31', 'E3', 'E3', 'E3',...",E10 E31 E31 E31 E3 E3 E3 E27 E29 E27 E29 E27 E...
1,1,27644297472372,Normal,"['E31', 'E31', 'E31', 'E10', 'E27', 'E29', 'E2...",E31 E31 E31 E10 E27 E29 E27 E29 E27 E29 E3 E3 ...
2,2,33380551139732,Normal,"['E9', 'E31', 'E31', 'E31', 'E3', 'E3', 'E3', ...",E9 E31 E31 E31 E3 E3 E3 E27 E29 E27 E29 E27 E2...
3,3,35641716531335,Normal,"['E8', 'E31', 'E31', 'E31', 'E27', 'E29', 'E27...",E8 E31 E31 E31 E27 E29 E27 E29 E3 E3 E27 E29 E...
4,4,40083153975957,Normal,"['E31', 'E8', 'E31', 'E31', 'E27', 'E29', 'E27...",E31 E8 E31 E31 E27 E29 E27 E29 E27 E29 E3 E3 E...


In [ ]:
data.Content_npl[0]

In [9]:
cv=CountVectorizer()
#data_cv=cv.fit_transform(data.Content_npl)
data_cv=cv.fit_transform(data.Event_id)
data_bag=pd.DataFrame(data_cv.toarray(),columns=cv.get_feature_names())


In [10]:
data_bag["Label"]=data.Label

In [11]:
data_dtm=data_bag

In [12]:
data_dtm.head()

,e0,e1,e10,e11,e12,e13,e14,e15,e16,e17,...,e39,e4,e40,e41,e5,e6,e7,e8,e9,Label
0,0,0,1,0,3,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,Normal
1,0,0,1,0,3,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,Normal
2,0,0,0,0,3,0,0,0,3,0,...,0,0,0,0,0,0,0,0,1,Normal
3,0,0,0,0,3,0,0,0,3,0,...,0,0,0,0,0,0,0,1,0,Normal
4,0,0,0,0,3,0,0,0,3,0,...,0,0,0,0,0,0,0,1,0,Normal


In [10]:
data_dtm.iloc[:, -1]
data_dtm.iloc[:, :-1]

,e0,e1,e10,e11,e12,e13,e14,e15,e16,e17,e18,e19,e2,e20,e21,e22,e23,e24,e25,e26,e27,e28,e29,e3,e30,e31,e32,e33,e34,e35,e36,e37,e38,e39,e4,e40,e41,e5,e6,e7,e8,e9
0,0,0,1,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,3,0,0,0,3,0,0,0,0,4,0,0,0,0,0,0,3,0,3,3,0,3,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575056,0,0,1,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
575057,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
575058,0,0,0,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
575059,0,0,1,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
# split the dataset
label_col = {'Normal': 1, 'Anomaly': 0}
# data_dtm.Label = [label_col[item] for item in data_dtm.Label]
y = data_dtm.iloc[:, -1]
X = data_dtm.iloc[:, :-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [12]:
X

,e0,e1,e10,e11,e12,e13,e14,e15,e16,e17,e18,e19,e2,e20,e21,e22,e23,e24,e25,e26,e27,e28,e29,e3,e30,e31,e32,e33,e34,e35,e36,e37,e38,e39,e4,e40,e41,e5,e6,e7,e8,e9
0,0,0,1,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,3,0,0,0,3,0,0,0,0,4,0,0,0,0,0,0,3,0,3,3,0,3,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575056,0,0,1,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
575057,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
575058,0,0,0,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
575059,0,0,1,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
# Train the model
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

# Prediction
y_pred = classifier.predict(X_test)

In [15]:
y_test

259307    Normal
355664    Normal
555412    Normal
278419    Normal
262819    Normal
           ...  
139729    Normal
553095    Normal
233705    Normal
44721     Normal
49872     Normal
Name: Label, Length: 230025, dtype: object

In [18]:
# Evaluate model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
df_confusion = pd.crosstab(y_test, y_pred)
df_confusion

              precision    recall  f1-score   support

     Anomaly       1.00      1.00      1.00      6633
      Normal       1.00      1.00      1.00    223392

    accuracy                           1.00    230025
   macro avg       1.00      1.00      1.00    230025
weighted avg       1.00      1.00      1.00    230025

0.9999043582219324


col_0,Anomaly,Normal
Label,,
Anomaly,6625,8
Normal,14,223378


***2. Bag of Words with Term Frequency-Inverse Document Frequency***

In [20]:
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

In [22]:
# Train the model
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

# Prediction
y_pred = classifier.predict(X_test)

In [18]:
# Evaluate model
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
df_confusion = pd.crosstab(y_test, y_pred)
df_confusion

[[  6706     11]
 [    17 223291]]
              precision    recall  f1-score   support

     Anomaly       1.00      1.00      1.00      6717
      Normal       1.00      1.00      1.00    223308

    accuracy                           1.00    230025
   macro avg       1.00      1.00      1.00    230025
weighted avg       1.00      1.00      1.00    230025

0.9998782741006412


col_0,Anomaly,Normal
Label,,
Anomaly,6706,11
Normal,17,223291


In [25]:
# Train the model
classifier = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
classifier.fit(X_train, y_train)

# Prediction
y_pred = classifier.predict(X_test)

[LibLinear]

In [26]:
# Evaluate model
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
df_confusion = pd.crosstab(y_test, y_pred)
df_confusion

[[  5031     19]
 [   137 167332]]
              precision    recall  f1-score   support

     Anomaly       0.97      1.00      0.98      5050
      Normal       1.00      1.00      1.00    167469

    accuracy                           1.00    172519
   macro avg       0.99      1.00      0.99    172519
weighted avg       1.00      1.00      1.00    172519

0.9990957517722686


col_0,Anomaly,Normal
Label,,
Anomaly,5031,19
Normal,137,167332


In [24]:
# Evaluate model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[  5031     19]
 [   137 167332]]
              precision    recall  f1-score   support

     Anomaly       0.97      1.00      0.98      5050
      Normal       1.00      1.00      1.00    167469

    accuracy                           1.00    172519
   macro avg       0.99      1.00      0.99    172519
weighted avg       1.00      1.00      1.00    172519

0.9990957517722686


***3.1 Doc2vec Log Event Analysis (Event Based)***

In [2]:
# import dataset
df=pd.read_csv('/Users/yuxuancui/Desktop/MDS/data586/project/rnn/unique_id.csv')

In [3]:
df

,Unnamed: 0,BlockId,Label,Event,Event_id
0,0,24807389905763,Normal,"['E10', 'E31', 'E31', 'E31', 'E3', 'E3', 'E3',...",E10 E31 E31 E31 E3 E3 E3 E27 E29 E27 E29 E27 E...
1,1,27644297472372,Normal,"['E31', 'E31', 'E31', 'E10', 'E27', 'E29', 'E2...",E31 E31 E31 E10 E27 E29 E27 E29 E27 E29 E3 E3 ...
2,2,33380551139732,Normal,"['E9', 'E31', 'E31', 'E31', 'E3', 'E3', 'E3', ...",E9 E31 E31 E31 E3 E3 E3 E27 E29 E27 E29 E27 E2...
3,3,35641716531335,Normal,"['E8', 'E31', 'E31', 'E31', 'E27', 'E29', 'E27...",E8 E31 E31 E31 E27 E29 E27 E29 E3 E3 E27 E29 E...
4,4,40083153975957,Normal,"['E31', 'E8', 'E31', 'E31', 'E27', 'E29', 'E27...",E31 E8 E31 E31 E27 E29 E27 E29 E27 E29 E3 E3 E...
...,...,...,...,...,...
575056,575056,9223281626658782772,Normal,"['E10', 'E31', 'E31', 'E31', 'E3', 'E3', 'E3',...",E10 E31 E31 E31 E3 E3 E3 E27 E29 E27 E29 E27 E...
575057,575057,9223296992671439323,Normal,"['E31', 'E31', 'E31', 'E8', 'E27', 'E29', 'E3'...",E31 E31 E31 E8 E27 E29 E3 E27 E29 E27 E29 E3 E3
575058,575058,9223337656441635326,Normal,"['E31', 'E31', 'E31', 'E8', 'E27', 'E29', 'E27...",E31 E31 E31 E8 E27 E29 E27 E29 E27 E29 E3 E3 E...
575059,575059,9223343671363935345,Normal,"['E31', 'E31', 'E31', 'E10', 'E27', 'E29', 'E2...",E31 E31 E31 E10 E27 E29 E27 E29 E27 E29 E3 E3 ...


In [4]:
# split the dataset
train, test = train_test_split(df, test_size=0.2, random_state=0)


In [5]:
train

,Unnamed: 0,BlockId,Label,Event,Event_id
461372,461372,7410049585842026434,Normal,"['E8', 'E31', 'E31', 'E31', 'E3', 'E3', 'E3', ...",E8 E31 E31 E31 E3 E3 E3 E27 E29 E27 E29 E27 E2...
556066,556066,8926175720539329280,Normal,"['E31', 'E31', 'E31', 'E8', 'E27', 'E29', 'E27...",E31 E31 E31 E8 E27 E29 E27 E29 E27 E29 E3 E3 E...
259706,259706,4177034920749953926,Normal,"['E8', 'E31', 'E31', 'E31', 'E27', 'E29', 'E27...",E8 E31 E31 E31 E27 E29 E27 E29 E27 E29 E3 E3 E...
148602,148602,2387726816865559953,Normal,"['E8', 'E31', 'E31', 'E31', 'E27', 'E29', 'E27...",E8 E31 E31 E31 E27 E29 E27 E29 E27 E29 E3 E3 E...
230052,230052,3700474212050631580,Normal,"['E31', 'E31', 'E31', 'E9', 'E27', 'E29', 'E27...",E31 E31 E31 E9 E27 E29 E27 E29 E27 E29 E3 E3 E...
...,...,...,...,...,...
359783,359783,5781885357792024476,Normal,"['E31', 'E8', 'E31', 'E31', 'E27', 'E29', 'E27...",E31 E8 E31 E31 E27 E29 E27 E29 E27 E29 E3 E3 E...
152315,152315,2446581588503680489,Normal,"['E8', 'E31', 'E31', 'E31', 'E3', 'E3', 'E27',...",E8 E31 E31 E31 E3 E3 E27 E29 E27 E29 E3 E27 E2...
117952,117952,1895842145726765448,Normal,"['E31', 'E31', 'E31', 'E10', 'E27', 'E29', 'E2...",E31 E31 E31 E10 E27 E29 E27 E29 E27 E29 E3 E3 ...
435829,435829,7006020402735781357,Normal,"['E9', 'E31', 'E31', 'E31', 'E27', 'E29', 'E27...",E9 E31 E31 E31 E27 E29 E27 E29 E27 E29 E3 E3 E...


In [6]:
train_tagged = train.apply(lambda r: TaggedDocument(words=r['Event'], tags=[r.Label]), axis=1)
test_tagged = test.apply(lambda r: TaggedDocument(words=r['Event'], tags=[r.Label]), axis=1)

In [7]:
train_tagged.values[30]

TaggedDocument(words="['E31', 'E31', 'E31', 'E8', 'E27', 'E29', 'E27', 'E29', 'E3', 'E27', 'E29', 'E3', 'E3']", tags=['Normal'])

In [8]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

INFO - 10:59:36: Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d300,n5,mc2,t3)', 'datetime': '2021-05-07T10:59:36.168698', 'gensim': '4.0.1', 'python': '3.8.3 (default, May 19 2020, 13:54:14) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.15.7-x86_64-i386-64bit', 'event': 'created'}
100%|██████████| 460048/460048 [00:00<00:00, 2585294.61it/s]
INFO - 10:59:36: collecting all words and their counts
WARNING - 10:59:36: Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
INFO - 10:59:36: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 10:59:36: PROGRESS: at example #10000, processed 1323969 words (8806112/s), 16 word types, 2 tags
INFO - 10:59:36: PROGRESS: at example #20000, processed 2650002 words (9217864/s), 16 word types, 2 tags
INFO - 10:59:36: PROGRESS: at example #30000, processed 3973368 words (8862098/s), 16 word types, 2 tags
INFO - 10:59:36: PROGRESS: at example #40000, processed 529292

In [9]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector([doc.words], steps=20)) for doc in sents])
    return targets, regressors


In [10]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)



In [13]:
np.save("X_train_e.npy",X_train)
np.save("X_test_e.npy",X_test)
np.save("Y_train_e.npy",y_train)
np.save("Y_test_e.npy",y_test)

In [ ]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

In [17]:
# Evaluate model
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))


Testing accuracy 0.9839757244833193
Testing F1 score: 0.9817081997191693
[[  1649   1666]
 [   177 111521]]
              precision    recall  f1-score   support

     Anomaly       0.90      0.50      0.64      3315
      Normal       0.99      1.00      0.99    111698

    accuracy                           0.98    115013
   macro avg       0.94      0.75      0.82    115013
weighted avg       0.98      0.98      0.98    115013

0.9839757244833193


In [14]:
# Train the model
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

# Prediction
y_pred = classifier.predict(X_test)

In [ ]:
# Evaluate model
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
df_confusion = pd.crosstab(y_test, y_pred)
df_confusion

***3.2 Doc2vec Log Event Analysis (Content Based)***

able to train the model with new conent 

In [4]:
log=pd.read_csv('/Users/yuxuancui/Desktop/MDS/data586/project/rnn/log_d2v.csv')

In [5]:
log.head()

,Unnamed: 0,Label,Content_npl,Full_Description
0,0,Normal,BLOCK NameSystem allocateBlock user root sortr...,"['BLOCK', 'NameSystem', 'allocateBlock', 'user..."
1,1,Normal,Receiving block blk src dest Receiving block b...,"['Receiving', 'block', 'blk', 'src', 'dest', '..."
2,2,Normal,BLOCK NameSystem allocateBlock user root randt...,"['BLOCK', 'NameSystem', 'allocateBlock', 'user..."
3,3,Normal,BLOCK NameSystem allocateBlock user root rand ...,"['BLOCK', 'NameSystem', 'allocateBlock', 'user..."
4,4,Normal,Receiving block blk src dest BLOCK NameSystem ...,"['Receiving', 'block', 'blk', 'src', 'dest', '..."


In [6]:
# split the dataset
train, test = train_test_split(log, test_size=0.2, random_state=0)

In [7]:
train_tagged = train.apply(lambda r: TaggedDocument(words=r['Full_Description'], tags=[r.Label]), axis=1)
test_tagged = test.apply(lambda r: TaggedDocument(words=r['Full_Description'], tags=[r.Label]), axis=1)

In [8]:
train_tagged.values[30]

TaggedDocument(words="['Receiving', 'block', 'blk', 'src', 'dest', 'Receiving', 'block', 'blk', 'src', 'dest', 'Receiving', 'block', 'blk', 'src', 'dest', 'BLOCK', 'NameSystem', 'allocateBlock', 'user', 'root', 'rand', 'temporary', 'task', 'm', 'part', 'blk', 'PacketResponder', 'for', 'block', 'blk', 'terminatingReceived', 'block', 'blk', 'of', 'size', 'from', 'PacketResponder', 'for', 'block', 'blk', 'terminatingReceived', 'block', 'blk', 'of', 'size', 'from', 'BLOCK', 'NameSystem', 'addStoredBlock', 'blockMap', 'updated', 'is', 'added', 'to', 'blk', 'size', 'PacketResponder', 'for', 'block', 'blk', 'terminatingReceived', 'block', 'blk', 'of', 'size', 'from', 'BLOCK', 'NameSystem', 'addStoredBlock', 'blockMap', 'updated', 'is', 'added', 'to', 'blk', 'size', 'BLOCK', 'NameSystem', 'addStoredBlock', 'blockMap', 'updated', 'is', 'added', 'to', 'blk', 'size']", tags=['Normal'])

In [9]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

INFO - 17:03:50: Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d300,n5,mc2,t3)', 'datetime': '2021-05-06T17:03:50.453213', 'gensim': '4.0.1', 'python': '3.8.3 (default, May 19 2020, 13:54:14) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.15.7-x86_64-i386-64bit', 'event': 'created'}
100%|██████████| 460048/460048 [00:00<00:00, 2726359.83it/s]
INFO - 17:03:50: collecting all words and their counts
WARNING - 17:03:50: Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
INFO - 17:03:50: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 17:03:51: PROGRESS: at example #10000, processed 13463190 words (10594218/s), 49 word types, 2 tags
INFO - 17:03:53: PROGRESS: at example #20000, processed 26936065 words (10770915/s), 49 word types, 2 tags
INFO - 17:03:54: PROGRESS: at example #30000, processed 40395464 words (10428796/s), 49 word types, 2 tags
INFO - 17:03:55: PROGRESS: at example #40000, processed 

In [13]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)


In [16]:
np.save("X_train_c.npy",X_train)
np.save("X_test_c.npy",X_test)
np.save("Y_train_c.npy",y_train)
np.save("Y_test_c.npy",y_test)

In [17]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

/Users/yuxuancui/opt/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
Testing accuracy 0.981219514315773
Testing F1 score: 0.9773657219044659


In [19]:

# Evaluate model
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))


[[  1256   2059]
 [   101 111597]]
              precision    recall  f1-score   support

     Anomaly       0.93      0.38      0.54      3315
      Normal       0.98      1.00      0.99    111698

    accuracy                           0.98    115013
   macro avg       0.95      0.69      0.76    115013
weighted avg       0.98      0.98      0.98    115013

0.981219514315773


In [20]:
# Train the model
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

# Prediction
y_pred = classifier.predict(X_test)

In [ ]:
# Evaluate model
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
